In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch_numopt
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

In [2]:
device = "cpu"

In [3]:
class Net(nn.Module):
    def __init__(self, input_size, device="cpu"):
        super().__init__()
        self.f1 = nn.Linear(input_size, 10, device=device)
        self.f2 = nn.Linear(10, 20, device=device)
        self.f3 = nn.Linear(20, 20, device=device)
        self.f4 = nn.Linear(20, 10, device=device)
        self.f5 = nn.Linear(10, 1, device=device)

        self.activation = nn.ReLU()
        # self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.f1(x))
        x = self.activation(self.f2(x))
        x = self.activation(self.f3(x))
        x = self.activation(self.f4(x))
        x = self.f5(x)

        return x

In [4]:
# X, y = load_diabetes(return_X_y=True, scaled=False)
# X, y = make_regression(n_samples=1000, n_features=100)
X, y = make_friedman1(n_samples=1000, noise=1e-2)

X_scaler = MinMaxScaler()
X = X_scaler.fit_transform(X)
X = torch.Tensor(X).to(device)

y_scaler = MinMaxScaler()
y = y_scaler.fit_transform(y.reshape((-1, 1)))
y = torch.Tensor(y).to(device)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

torch_data = TensorDataset(X_train, y_train)
data_loader = DataLoader(torch_data, batch_size=1000)

torch.Size([800, 10]) torch.Size([800, 1])
torch.Size([200, 10]) torch.Size([200, 1])


In [6]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = torch_numopt.AGD(
    params=model.parameters(),
    model=model,
    lr=1,
    mu=0.001,
    mu_dec=0.1,
    use_diagonal=False,
    c1=1e-4,
    tau=0.5,
    line_search_method="backtrack",
    line_search_cond="armijo",
)

all_loss = []
patience = 0
max_patience = 10
for epoch in range(100):
    print("epoch: ", epoch, end="")
    all_loss.append(0)
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch] += loss

    all_loss[epoch] /= len(data_loader)

    if epoch > 0 and all_loss[epoch - 1] <= all_loss[epoch]:
        patience -= 1
    else:
        patience = max_patience

    print(", loss: {}".format(all_loss[epoch].detach().cpu().numpy().item()))

    if patience <= 0:
        break

epoch:  0, loss: 0.12176299840211868
epoch:  1, loss: 0.08887650817632675
epoch:  2, loss: 0.030889704823493958
epoch:  3, loss: 0.029903989285230637
epoch:  4, loss: 0.012602063827216625
epoch:  5, loss: 0.009692499414086342
epoch:  6, loss: 0.009599948301911354
epoch:  7, loss: 0.007879713550209999
epoch:  8, loss: 0.00784590095281601
epoch:  9, loss: 0.006391721777617931
epoch:  10, loss: 0.006210894789546728
epoch:  11, loss: 0.005086940713226795
epoch:  12, loss: 0.004753451328724623
epoch:  13, loss: 0.004552618134766817
epoch:  14, loss: 0.004539756570011377
epoch:  15, loss: 0.00389763037674129
epoch:  16, loss: 0.0037189736030995846
epoch:  17, loss: 0.0031598759815096855
epoch:  18, loss: 0.003077868605032563
epoch:  19, loss: 0.0025409169029444456
epoch:  20, loss: 0.002379290061071515
epoch:  21, loss: 0.00228571193292737
epoch:  22, loss: 0.0022808292414993048
epoch:  23, loss: 0.0019331626826897264
epoch:  24, loss: 0.0019268589094281197
epoch:  25, loss: 0.00164633081294

In [7]:
pred_train = model.forward(X_train).detach()
pred_test = model.forward(X_test).detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train)}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test)}")

Train metrics: R2 = 0.9908385552463271
Test metrics:  R2 = 0.9872556494552939


In [8]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = torch_numopt.AGD(
    params=model.parameters(),
    model=model,
    lr=1,
    mu=0.001,
    mu_dec=0.1,
    use_diagonal=True,
    c1=1e-4,
    tau=0.1,
    line_search_method="backtrack",
    line_search_cond="armijo",
)

all_loss = []
patience = 0
max_patience = 10
for epoch in range(100):
    print("epoch: ", epoch, end="")
    all_loss.append(0)
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch] += loss

    all_loss[epoch] /= len(data_loader)

    if epoch > 0 and all_loss[epoch - 1] <= all_loss[epoch]:
        patience -= 1
    else:
        patience = max_patience

    print(", loss: {}".format(all_loss[epoch].detach().cpu().numpy().item()))

    if patience <= 0:
        break

epoch:  0, loss: 0.31840285658836365
epoch:  1, loss: 0.0384487621486187
epoch:  2, loss: 0.03130770102143288
epoch:  3, loss: 0.020043374970555305
epoch:  4, loss: 0.013425788842141628
epoch:  5, loss: 0.011436599306762218
epoch:  6, loss: 0.010700805112719536
epoch:  7, loss: 0.010106268338859081
epoch:  8, loss: 0.009780503809452057
epoch:  9, loss: 0.009556412696838379
epoch:  10, loss: 0.009382104501128197
epoch:  11, loss: 0.009239071980118752
epoch:  12, loss: 0.009087963961064816
epoch:  13, loss: 0.008752881549298763
epoch:  14, loss: 0.008117079734802246
epoch:  15, loss: 0.007579416502267122
epoch:  16, loss: 0.007348878309130669
epoch:  17, loss: 0.006920448504388332
epoch:  18, loss: 0.006698453798890114
epoch:  19, loss: 0.006529805716127157
epoch:  20, loss: 0.006390022113919258
epoch:  21, loss: 0.006274827755987644
epoch:  22, loss: 0.0061707948334515095
epoch:  23, loss: 0.006086251698434353
epoch:  24, loss: 0.005593189503997564
epoch:  25, loss: 0.005515162833034992

In [9]:
pred_train = model.forward(X_train).detach()
pred_test = model.forward(X_test).detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train)}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test)}")

Train metrics: R2 = 0.9035798914664308
Test metrics:  R2 = 0.8818791135552208
